In [ ]:
#Description: Build a book recomendation system

In [1]:
#Load the data
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
#Load the data
from google.colab import files
files.upload()

Saving grbooksds.csv to grbooksds.csv


{'grbooksds.csv': b'book_id,title,author,isbn,isbn13,asin,num_pages,avg_rating,num_ratings,year_pub_ed,text_review_count,genre,publisher,language,title_URL\r\n1,Nearly All the Men in Lagos Are Mad ,"Kuku, Damilare",,9.79E+12,,254,4.01,2919,2021,457,Short Stories,Masobe Books,English,https://www.goodreads.com/book/show/58120777-nearly-all-the-men-in-lagos-are-mad\r\n2,Open Water ,"Nelson, Caleb Azumah",241448778,9.78E+12,241448778,145,4.09,46508,2021,7502,Fiction,VIKIN,English,https://www.goodreads.com/book/show/53414230-open-water\r\n3,"Love in Colour: Mythical Tales from Around the World, Retold ","Babalola, Bolu",1472268865,9.78E+12,1472268865,304,3.99,13298,2020,2296,Romance,Headline,English,https://www.goodreads.com/book/show/51102251-love-in-colour\r\n4,"The Darkest Legacy\n (The Darkest Minds, #4)","Bracken, Alexandra",136802324X,9.78E+12,136802324X,569,4.13,10404,2018,1133,Dystopia,Disney Hyperion,English,https://www.goodreads.com/book/show/37688046-the-darkest-legacy\r\n5,"In t

In [3]:
#Get file path
import os
file_path = os.path.abspath('grbooksds.csv')
print(file_path)

/content/grbooksds.csv


In [4]:
#Replace file path
file_path = 'grbooksds.csv'

In [5]:
#Store the data
df = pd.read_csv(file_path, encoding='unicode_escape', error_bad_lines=False)

<ipython-input-5-ed42f0521f53>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(file_path, encoding='unicode_escape', error_bad_lines=False)


In [6]:
#Show the data
df

,book_id,title,author,isbn,isbn13,asin,num_pages,avg_rating,num_ratings,year_pub_ed,text_review_count,genre,publisher,language,title_URL
0,1,Nearly All the Men in Lagos Are Mad,"Kuku, Damilare",NaN,9.790000e+12,NaN,254,4.01,2919,2021,457,Short Stories,Masobe Books,English,https://www.goodreads.com/book/show/58120777-n...
1,2,Open Water,"Nelson, Caleb Azumah",241448778,9.780000e+12,241448778,145,4.09,46508,2021,7502,Fiction,VIKIN,English,https://www.goodreads.com/book/show/53414230-o...
2,3,Love in Colour: Mythical Tales from Around the...,"Babalola, Bolu",1472268865,9.780000e+12,1472268865,304,3.99,13298,2020,2296,Romance,Headline,English,https://www.goodreads.com/book/show/51102251-l...
3,4,"The Darkest Legacy\n (The Darkest Minds, #4)","Bracken, Alexandra",136802324X,9.780000e+12,136802324X,569,4.13,10404,2018,1133,Dystopia,Disney Hyperion,English,https://www.goodreads.com/book/show/37688046-t...
4,5,"In the Afterlight\n (The Darkest Minds, #3)","Bracken, Alexandra",1423157524,9.780000e+12,1423157524,535,4.25,60795,2014,5853,Dystopia,Disney-Hyperion,English,https://www.goodreads.com/book/show/16150831-i...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746,747,"Hunting Adeline\n (Cat and Mouse Duet, #2)","Carlton, H. D.",NaN,NaN,B096BKV268,531,4.19,133524,2022,13765,Romance,Hailey Carlton,English,https://www.goodreads.com/book/show/59050133-h...
747,748,"Haunting Adeline\n (Cat and Mouse Duet, #1)","Carlton, H. D.",NaN,NaN,B09CLVJJ77,551,4.05,204351,2021,25408,Romance,Hailey Carlton,English,https://www.goodreads.com/book/show/58763686-h...
748,749,"Final Offer\n (Dreamland Billionaires, #3)","Asher, Lauren",172827222X,9.780000e+12,172827222X,583,4.30,120517,2023,14647,Romance,Bloom Books,English,https://www.goodreads.com/book/show/61681786-f...
749,750,Terms and Conditions\n (Dreamland Billionaires...,"Asher, Lauren",NaN,NaN,B09PTXQL3J,494,4.19,210532,2022,21488,Romance,"Little, Brown Book Group",English,https://www.goodreads.com/book/show/86767939-t...


In [7]:
#Create list of columns to keep
columns =['title', 'author', 'genre', 'publisher']

In [8]:
#Create a function to combine the important/relevant columns
def combine_features(data):
  features = []
  for i in range(0, data.shape[0]):
    features.append( data['title'][i] + ' '+data['author'][i] +' '+ data['genre'][i] +' '+ data['publisher'][i])

  return features

In [9]:
#Create a column to store the combined features
df['combined_features']=combine_features(df)

#Show the data
df

,book_id,title,author,isbn,isbn13,asin,num_pages,avg_rating,num_ratings,year_pub_ed,text_review_count,genre,publisher,language,title_URL,combined_features
0,1,Nearly All the Men in Lagos Are Mad,"Kuku, Damilare",NaN,9.790000e+12,NaN,254,4.01,2919,2021,457,Short Stories,Masobe Books,English,https://www.goodreads.com/book/show/58120777-n...,"Nearly All the Men in Lagos Are Mad Kuku, Dam..."
1,2,Open Water,"Nelson, Caleb Azumah",241448778,9.780000e+12,241448778,145,4.09,46508,2021,7502,Fiction,VIKIN,English,https://www.goodreads.com/book/show/53414230-o...,"Open Water Nelson, Caleb Azumah Fiction VIKIN"
2,3,Love in Colour: Mythical Tales from Around the...,"Babalola, Bolu",1472268865,9.780000e+12,1472268865,304,3.99,13298,2020,2296,Romance,Headline,English,https://www.goodreads.com/book/show/51102251-l...,Love in Colour: Mythical Tales from Around the...
3,4,"The Darkest Legacy\n (The Darkest Minds, #4)","Bracken, Alexandra",136802324X,9.780000e+12,136802324X,569,4.13,10404,2018,1133,Dystopia,Disney Hyperion,English,https://www.goodreads.com/book/show/37688046-t...,"The Darkest Legacy\n (The Darkest Minds, #4) B..."
4,5,"In the Afterlight\n (The Darkest Minds, #3)","Bracken, Alexandra",1423157524,9.780000e+12,1423157524,535,4.25,60795,2014,5853,Dystopia,Disney-Hyperion,English,https://www.goodreads.com/book/show/16150831-i...,"In the Afterlight\n (The Darkest Minds, #3) Br..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746,747,"Hunting Adeline\n (Cat and Mouse Duet, #2)","Carlton, H. D.",NaN,NaN,B096BKV268,531,4.19,133524,2022,13765,Romance,Hailey Carlton,English,https://www.goodreads.com/book/show/59050133-h...,"Hunting Adeline\n (Cat and Mouse Duet, #2) Car..."
747,748,"Haunting Adeline\n (Cat and Mouse Duet, #1)","Carlton, H. D.",NaN,NaN,B09CLVJJ77,551,4.05,204351,2021,25408,Romance,Hailey Carlton,English,https://www.goodreads.com/book/show/58763686-h...,"Haunting Adeline\n (Cat and Mouse Duet, #1) Ca..."
748,749,"Final Offer\n (Dreamland Billionaires, #3)","Asher, Lauren",172827222X,9.780000e+12,172827222X,583,4.30,120517,2023,14647,Romance,Bloom Books,English,https://www.goodreads.com/book/show/61681786-f...,"Final Offer\n (Dreamland Billionaires, #3) Ash..."
749,750,Terms and Conditions\n (Dreamland Billionaires...,"Asher, Lauren",NaN,NaN,B09PTXQL3J,494,4.19,210532,2022,21488,Romance,"Little, Brown Book Group",English,https://www.goodreads.com/book/show/86767939-t...,Terms and Conditions\n (Dreamland Billionaires...


In [11]:
#Convert the text from the new column to a matrix of word counts
cm = CountVectorizer().fit_transform(df['combined_features'])

In [12]:
#Get the cosine similarity matrix from the count matrix
cs = cosine_similarity(cm)

#Print the scores
print(cs)

[[1.         0.         0.14285714 ... 0.08908708 0.         0.07715167]
 [0.         1.         0.         ... 0.         0.         0.        ]
 [0.14285714 0.         1.         ... 0.08908708 0.07715167 0.15430335]
 ...
 [0.08908708 0.         0.08908708 ... 1.         0.48112522 0.48112522]
 [0.         0.         0.07715167 ... 0.48112522 1.         0.75      ]
 [0.07715167 0.         0.15430335 ... 0.48112522 0.75       1.        ]]


In [50]:
# Get the title of the book the user inputs
user_input_title = input("Enter the title of a book you like: ")

Enter the title of a book you like: Hunting Adeline


In [51]:
# Find the book id for the book that the user inputs
matching_books = df[df['title'].str.contains(user_input_title, case=False)]

if matching_books.empty:
    print("No matching books found.")
else:
    book_id = matching_books.index[0]

# Show the book id
book_id

746

In [52]:
#Create a list of tuples in the form (book_id, similarity score)
scores = list(enumerate(cs[book_id]))
print(scores)

[(0, 0.0), (1, 0.0), (2, 0.07715167498104597), (3, 0.0), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.1091089451179962), (9, 0.1091089451179962), (10, 0.09622504486493763), (11, 0.08333333333333336), (12, 0.10206207261596575), (13, 0.10206207261596575), (14, 0.1091089451179962), (15, 0.11785113019775795), (16, 0.07715167498104597), (17, 0.0), (18, 0.0), (19, 0.08333333333333336), (20, 0.1091089451179962), (21, 0.052704627669472995), (22, 0.0), (23, 0.0912870929175277), (24, 0.0), (25, 0.0), (26, 0.0), (27, 0.0), (28, 0.0), (29, 0.0), (30, 0.09622504486493763), (31, 0.0), (32, 0.0), (33, 0.0), (34, 0.0), (35, 0.0), (36, 0.0), (37, 0.0), (38, 0.0), (39, 0.0), (40, 0.0), (41, 0.0), (42, 0.0), (43, 0.07715167498104597), (44, 0.06622661785325219), (45, 0.08703882797784893), (46, 0.07453559924999299), (47, 0.08333333333333336), (48, 0.0912870929175277), (49, 0.10206207261596575), (50, 0.10206207261596575), (51, 0.10206207261596575), (52, 0.10206207261596575), (53, 0.0), (54, 0.0), (55, 0.11

In [53]:
#Sort the list of similar books in descending order
sorted_scores = sorted(scores, key= lambda x:x[1], reverse= True)
sorted_scores = sorted_scores[1:]

#Show the sorted scores
sorted_scores

[(747, 0.916666666666667),
 (664, 0.5773502691896258),
 (210, 0.2357022603955159),
 (392, 0.2041241452319315),
 (121, 0.18898223650461365),
 (687, 0.18898223650461365),
 (289, 0.1825741858350554),
 (486, 0.1825741858350554),
 (492, 0.17407765595569785),
 (749, 0.1666666666666667),
 (491, 0.14907119849998599),
 (597, 0.14433756729740646),
 (622, 0.140028008402801),
 (467, 0.13245323570650439),
 (673, 0.13245323570650439),
 (571, 0.12909944487358058),
 (675, 0.12598815766974242),
 (727, 0.12598815766974242),
 (728, 0.12598815766974242),
 (338, 0.12309149097933274),
 (15, 0.11785113019775795),
 (55, 0.11785113019775795),
 (433, 0.11785113019775795),
 (620, 0.11785113019775795),
 (624, 0.11785113019775795),
 (628, 0.11785113019775795),
 (665, 0.11785113019775795),
 (725, 0.11785113019775795),
 (745, 0.11785113019775795),
 (734, 0.11547005383792518),
 (8, 0.1091089451179962),
 (9, 0.1091089451179962),
 (14, 0.1091089451179962),
 (20, 0.1091089451179962),
 (198, 0.1091089451179962),
 (267, 0

In [54]:
# Print the first 5 recommended books
print(f"Top 5 recommendations for '{user_input_title}':")
for i, (book_id, similarity_score) in enumerate(sorted_scores[:5]):
    recommended_book = df.loc[book_id, 'title']
    print(f"Recommendation {i+1}: {recommended_book} (Similarity Score: {similarity_score:.4f})")

Top 5 recommendations for 'Hunting Adeline':
Recommendation 1: Haunting Adeline
 (Cat and Mouse Duet, #1) (Similarity Score: 0.9167)
Recommendation 2: Does It Hurt?  (Similarity Score: 0.5774)
Recommendation 3: Meet Cute  (Similarity Score: 0.2357)
Recommendation 4: Exes and O's
 (The Influencer, #2) (Similarity Score: 0.2041)
Recommendation 5: Ruin and Rising
 (The Shadow and Bone Trilogy, #3) (Similarity Score: 0.1890)
